# Examen Intra Automne 2020

## Question 1 (5 pts)

Est-il possible d'avoir une solution optimale qui ne soit pas une solution de base?

**Oui**. Le théorème fondamental de la programmation linéaire nous dit que si une solution optimale existe, alors une solution de base optimale existe, mais il ne nous dit pas que toute solution optimale est également une solution de base.

Prenons par exemple le programme
\begin{align*}
\min_x \ & x_1 + x_2 \\
\mbox{t.q. } & x_1 + x_2 = 1 \\
& x_1 \geq 0, x_2 \geq 0.
\end{align*}
Alors toute combinaison convexe des 2 solutions de base optimales $(1,0)$ et $(0,1)$ est également solution optimale. Ainsi, $(0.5, 0.5)$ est une solution optimale, mais n'est pas une solution de base.

## Question 2 (5 pts)

Dans la procédure du simplexe, le choix de la variable sortante se fait toujours en
considérant du coût réduit le plus négatif.

**Non**. L'énoncé a ici deux erreurs. La première est le choix de la variable. C'est le variable entrante, non sortante, qui s'opère en regardant les coûts réduits négatifs. De plus, n'importe quel variable avec un coût réduit négatif est candidate pour entrer dans la base, pas seulement celle avec le coût réduit le plus négatif.

Soit le programme
\begin{align*}
\min_x \ & c^Tx \\
\mbox{t.q. } & Ax = b \\
& x \geq 0.
\end{align*}
Le choix de la variable entrante $x_j$ se fait en choisissant $j \in \arg\min \{ y_{ik}/y_{i0} \,|\, y_{ik} > 0, \ 1 \leq i \leq m \}$, où $y_0 = B^{-1}b$, $y_k = B^{-1}a_k$, avec $B$ la base courante et $a_k$ la $k^e$ colonne de la matrice de contraintes $A$. $y_{ik}$ représente la $i^e$ composante de $y_k$.

## Question 3 (30 pts)

Soit le problème
\begin{align*}
\min\ & x_1 - x_2 \\
\mbox{tel que }  & 2x_1 + x_2 \geq 2 \\
& x_1 + 3x_2 \leq 3  \\
& x_1, x_2 \geq 0
\end{align*}

 - Résolvez-le (selon la méthode de votre choix).
 - Donnez la base optimale et son inverse.
 - Formulez le problème dual.

Commençons par résoudre le problème. Comme nous pouvons utiliser la méthode de notre choix, nous allons considérer une résolution numérique et une résolution manuelle. Pour l'approche numérique, nous utiliserons la librairie `JuMP` et le solveur `Clp`.

In [9]:
using LinearAlgebra, Clp, JuMP

m = Model(Clp.Optimizer)
@variable(m, x[1:2] >= 0)
@constraint(m, 2x[1] + x[2] >= 2)
@constraint(m, x[1] + 3x[2] <= 3)

@objective(m, Min, x[1] - x[2])

println(m)

Min x[1] - x[2]
Subject to
 2 x[1] + x[2] >= 2.0
 x[1] + 3 x[2] <= 3.0
 x[1] >= 0.0
 x[2] >= 0.0



In [10]:
optimize!(m)

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value -0.2
Coin0511I After Postsolve, objective -0.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -0.2 - 0 iterations time 0.002, Presolve 0.00


La solution est

In [11]:
value.(x)

2-element Array{Float64,1}:
 0.6
 0.7999999999999999

Nous pouvons confirmer cete value en résolvant le programme à la main.

Mettons tout d'abord le problème sous forme standard, en introduisant une variable de surplus, $x_3$, dans la première contrainte et une variable d'écart, $x_4$, dans la seconde:
\begin{align*}
\min\ & x_1 - x_2 \\
\mbox{tel que }  & 2x_1 + x_2 - x_3 = 2 \\
& x_1 + 3x_2 + x_4 = 3  \\
& x_1, x_2, x_3, x_4 \geq 0
\end{align*}
La variable $x_4$ est la seule variable isolée du problème.
La problème ne se présente pas sous forme canonique, et nous ne pouvons identifier directement une base réalisable.
Nous allons procéder avec une méthode à deux phases.

### Phase 1

Comme $x_4$ est déjà isolée, il suffit d'ajouter une variable artificielle à la première contrainte, menant au problème artificiel
\begin{align*}
\min\ & y \\
\mbox{tel que }  & 2x_1 + x_2 - x_3 + y = 2 \\
& x_1 + 3x_2 + x_4 = 3  \\
& x_1, x_2, x_3, x_4, y \geq 0
\end{align*}
Nous allons le résoudre par la méthode du simplexe sous forme tableau.
Le tableau initial donne
$$
\begin{matrix}
2 & 1 & -1 & 0 & 1 & 2 \\
1 & 3 & 0 & 1 & 0 & 3 \\
0 & 0 & 0 & 0 & 1 & 0
\end{matrix}
$$
Nous devons annuler le coût réduit associé à $y$, ce qui donne
$$
\begin{matrix}
2 & 1 & -1 & 0 & 1 & 2 \\
1 & 3 & 0 & 1 & 0 & 3 \\
-2 & -1 & 1 & 0 & 0 & -2
\end{matrix}
$$
Nous sommes en présence de deux coûts réduits négatifs.
En utilisant la règle empirique du cours réduit le plus négatif, nous décidons de faire entrer $x_1$ dans la base.
En comparant la première colonne de la matrice de contraintes et le terme de droite, nous voyons que $y$ sort de la base.
Ceci donne
$$
\begin{matrix}
1 & \frac{1}{2} & -\frac{1}{2} & 0 & \frac{1}{2} & 1 \\
0 & \frac{5}{2} & \frac{1}{2} & 1 & -\frac{1}{2} & 2 \\
0 & 0 & 0 & 0 & 1 & 0
\end{matrix}
$$
Il n'y a plus de coût réduit négatif, nous avons terminé la résolution du problème artificiel, et la variable artificielle est à présent hors base et donc nulle. Par conséquent, le problème est réalisable et nous pouvons passer à la phase 2.

### Phase 2

Nous partons du tableau du simplexe obtenu à la fin de la phase précédente, en incluant cette fois les coefficients de la fonction objectif dans la ligne des coûts.
$$
\begin{matrix}
1 & \frac{1}{2} & -\frac{1}{2} & 0 & 1 \\
0 & \frac{5}{2} & \frac{1}{2} & 1 & 2 \\
1 & -1 & 0 & 0 & 0
\end{matrix}
$$
En annulant les coûts réduits des variables de base, nous obtenons
$$
\begin{matrix}
1 & \frac{1}{2} & -\frac{1}{2} & 0 & 1 \\
0 & \frac{5}{2} & \frac{1}{2} & 1 & 2 \\
0 & -\frac{3}{2} & \frac{1}{2} & 0  & -1
\end{matrix}
$$
À partir de là, nous pouvons appliquer le simplexe classique, ce qui donne l'itération
$$
\begin{matrix}
1 & 0 & -\frac{3}{5} & -\frac{1}{5} & \frac{3}{5} \\
0 & 1 & \frac{1}{5} & \frac{2}{5} & \frac{4}{5} \\
0 & 0 & \frac{4}{5} & \frac{3}{5}  & \frac{1}{5}
\end{matrix}
$$
Il n'y a plus de coût réduit négatif, aussi nous avons convergé et la solution optimale est $x_1 = \frac{3}{5}$, $x_2 = \frac{4}{5}$, $x_3 = 0$ et $x_4 = 0$. La valeur optimale correspondante est $-\frac{1}{5}$.

Nous retrouvons (heureusement!) la solution obtenue avec la résolution numérique.

La base optimale est constituée des colonnes associées à $x_1$ et $x_2$, soit
$$
B = \begin{pmatrix}
2 & 1 \\ 1 & 3
\end{pmatrix}
$$
Nous pouvons directement lire l'inverse de $B$ du dernier tableau, comme les troisième et quatrième colonnes de la matrice initiale des contraintes donnent la matrice
$$
\begin{pmatrix}
-1 & 0 \\
0 & 1
\end{pmatrix}
$$
qui multiplié par elle-même donne la matrice identité.
Nous partons dès lors des colonnes associées à $x_3$ et $x_4$, qui correspondent à une itération du simplexe à
$$
B^{-1}
\begin{pmatrix}
-1 & 0 \\
0 & 1
\end{pmatrix}
$$
et en la multipliant par $\begin{pmatrix}
	-1 & 0 \\
	0 & 1
\end{pmatrix}$, i.e. en prenant l'opposé de la première colonne, donne $B^{-1}$.
Ainsi,
$$
B^{-1} =
\begin{pmatrix}
\frac{3}{5} & -\frac{1}{5} \\ -\frac{1}{5} & \frac{2}{5}
\end{pmatrix}
$$

Nous pouvons le vérifier numériquement, d'autant que nous n'avons pas accès au tableau du simplexe avec l'approche numérique.

In [12]:
B = [ 2 1 ; 1 3 ]
inv(B)

2×2 Array{Float64,2}:
  0.6  -0.2
 -0.2   0.4

Le problème dual est
\begin{align*}
\max\ & 2\lambda_1 + 3\lambda_2 \\
\mbox{tel que }
& 2\lambda_1 + \lambda_2 \leq 1 \\
& \lambda_1 + 3\lambda_2 \leq -1  \\
& \lambda_1 \geq 0 \\
& \lambda_2 \leq 0
\end{align*}
Il est à noter ici que $\lambda_1$ et $\lambda_2$ sont de signes opposés, comme dans le primal, les sens des inégalités des deux contraintes sont contraires.

## Question 4 (30 pts)

En utilisant la forme **révisée** du simplexe, trouvez une solution réalisable au système suivant:
\begin{align*}
x_1 + 2x_2 - x_3 + x_4 &= 3 \\
2x_1 + 4x_2 + x_3 + 2x_4 &= 12.
\end{align*}

Veuillez présenter chaque itération, en explicitant comment est choisi le pivot, et en mettant à jour le tableau du simplexe révisé à l'aide de Julia.

Nous devons tout d'abord reformuler le système linéaire comme un problème d'optimisation linéaire, tel quel demandé dans l'énoncé. Si le système linéaire admet une solution $x \geq 0$, celle-ci peut être obtenue directement avec la Phase I du simplexe, ou en d'autres termes, en résolvant le programme linéaire obtenu en introduisant les variables artificielles $x_5$ et $x_6$:
\begin{align*}
\min_{x} \ & x_5 + x_6 \\
\mbox{t.q. } & x_1 + 2x_2 - x_3 + x_4 + x_5 = 3 \\
& 2x_1 + 4x_2 + x_3 + 2x_4 + x_6 = 12 \\
& x \geq 0.
\end{align*}
Nous allons le résoudre avec le simplexe révisé, comme exigé.

Afin de faciliter les calculs avec Julia, nous introduisons la matrice $A$ et les vecteurs $c$ et $b$.

In [33]:
A = [ 1 2 -1 4 1 0 ; 2 4 1 2 0 1 ]
b = [ 3 ; 12 ]
c = [ 0 ; 0 ; 0 ; 0 ; 1 ; 1]

6-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1

Le premier tableau est
$$
\begin{matrix}
& x_5 & x_6 & b \\
x_5 & 1 & 0 & 3 \\
x_6 & 0 & 1 & 12 \\
\end{matrix}
$$
Calculons les coûts réduits. Pour ce faire, nous prenons la base initiale qui n'est autre que la matrice identité et calculons le vecteur $\lambda$ associé, ainsi que le vecteur des coûts réduits. Nous organisons ici les calculs pour avoir des vecteurs colonnes au lieu de vecteurs lignes, en ajoutant des opérations de transposition.

In [106]:
B = zeros(2,2)+I
Binv = B
λ = -transpose(Binv)*c[5:6]
r = c+transpose(A)*λ

6-element Array{Float64,1}:
 -3.0
 -6.0
  0.0
 -3.0
  0.0
  0.0

Nous voyons qu'il a trois coûts réduits négatifs. En choisissant le coût le plus négatif, nous faisons entrer $x_2$. La colonne correspondante dans le nouveau tableau du simplexe sera

In [107]:
y = Binv*A[:,2]

2-element Array{Float64,1}:
 2.0
 4.0

Ceci mène au tableau
$$
\begin{matrix}
& x_5 & x_6 & b & y \\
x_5 & 1 & 0 & 3 & 2 \\
x_6 & 0 & 1 & 12 & 4 \\
\end{matrix}
$$
Nous choissons le pivot en divisant élément par élément $b$ par $y$:

In [108]:
b./y

2-element Array{Float64,1}:
 1.5
 3.0

La variable de sortie correspond à pivoter sur la première ligne ce qui conduit à faire sortir $x_5$ de la base. Nous allons synthétiser le tableau du simplexe révisé en stockant les deux dernières colonnes de $A$, $b$ et $y$, et nous effectuerons le pivot sur cette matrice.

In [109]:
R = [A[:,5:6] b y]

2×4 Array{Float64,2}:
 1.0  0.0   3.0  2.0
 0.0  1.0  12.0  4.0

Ceci permettrant de toujour pivoter sur la dernière colonne, mais nous devrons choisir la ligne du pivot. Nous automatisons le pivot avec la fonction

In [110]:
function pivot!(A::Matrix, i::Int, j::Int)
    (m, n) = size(A)
    A[i, :] = A[i, :] / A[i, j]
    for k in 1:m
        if k != i
            A[k, :] -= A[i, :]*A[k, j]
        end
    end
    return A
end

pivot! (generic function with 1 method)

Muni de cette fonction, il devient aisé de pivoter, ce qui donner dans notre cas

In [112]:
pivot!(R, 1, 4)

2×4 Array{Float64,2}:
  0.5  0.0  1.5  1.0
 -2.0  1.0  6.0  0.0

Cela donne immédiatement le nouveau tableau du simplexe révisé
$$
\begin{matrix}
& x_5 & x_6 & b \\
x_2 & 0.5 & 0 & 1.5 \\
x_6 & -2 & 1 & 6 \\
\end{matrix}
$$

La nouvelle matrice inverse se lit en prenant les deux premières colonnes de $R$ et nous calculons les coûts réduits comme

In [114]:
r = c-transpose(A)*(transpose(R[:,1:2])*c[[2,6]])

6-element Array{Float64,1}:
  0.0
  0.0
 -3.0
  0.0
  3.0
  0.0

Nous pouvons davantage automatiser le code avec la fonction reducedcosts.

In [121]:
function reducedcosts(c::Vector, A::Matrix, R::Matrix, idx::Vector)
    (m,n) = size(A)
    return c-transpose(A)*(transpose(R[:,1:m])*c[idx])
end

reducedcosts (generic function with 1 method)

In [122]:
r = reducedcosts(c,A,R,[2,6])

6-element Array{Float64,1}:
  0.0
  0.0
 -3.0
  0.0
  3.0
  0.0

À nouveau, nous trouvons un coût réduit négatif, ce qui conduit à faire entrer $x_3$ dans la base. Nous calculons le vecteur $y_3 = B^{-1}$, que nous intégrons directement comme dernière colonne de $R$.

In [123]:
R[:,4] = R[:,1:2]*A[:,3]

2-element Array{Float64,1}:
 -0.5
  3.0

Nous sélectionnons à nouveau le pivot en divisant composante par composante la colonne du terme de droite avec la dernière colonne de $R$.

In [124]:
R[:,3]./R[:,4]

2-element Array{Float64,1}:
 -3.0
  2.0

Comme nous ne devons considérons que les termes non-négatifs (et finis), cela revient à choisir le second élément, autrement dit $x_6$ sort de la base. Nous pivotons donc sur la deuxième ligne pour obtenir le nouveau tableau

In [125]:
pivot!(R, 2, 4)

2×4 Array{Float64,2}:
  0.166667  0.166667  2.5  0.0
 -0.666667  0.333333  2.0  1.0

Cela donne immédiatement le tableau du simplexe révisé
$$
\begin{matrix}
& x_5 & x_6 & b \\
x_2 & 1/6 & 1/6 & 2.5 \\
x_3 & -2/3 & 1/3 & 2 \\
\end{matrix}
$$

Les coûts réduits deviennent

In [128]:
r = reducedcosts(c,A,R,[2,3])

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0

Il n'y a plus de coût réduit négatif, aussi avons-nous résolu notre problème artificiel. Comme $x_5 = x_6 = 0$, nous avons trouvé une solution réalisable au système d'équations initial, avec $x_1 = x_4 = 0$, $x_2 = 2.5$ et $x_3 = 2$.

## Question 5 (30 pts)

Considérons le problème
\begin{align*}
\min_x \ & z = 2x_1 + 4x_2 + x_3  + x_4 \\
\mbox{sujet à }
& x_1 + 3x_2 + x_4 \leq 4 \\
& 2x_1 +  x_2 + x_4 \leq 3 \\
& x_2 + 4x_3 + x_4 \leq 3 \\
& x_j \geq 0, j = 1,2,3,4.
\end{align*}

 - Est-il nécessaire d'employer une Phase I pour initialiser le simplexe? Identifiez la solution et la base optimales.
 - De combien peut-on changer $b = (4, 3, 3)$ sans changer de base optimale?
 - De combien peut-on changer $c = (2, 4, 1, 1)$ sans changer de base optimale?
 - Que  se  passe-t-il au  niveau du coût  optimal (valeur optimale de $z$ pour de petits changements dans $b$? Et pour de petits changements dans c?

Il n'est pas nécessaire d'employer ici une phase I comme la mise sous forme standard du problème donne directement un système sous forme canonique:
Considérons le problème
\begin{align*}
\min_x \ & z = 2x_1 + 4x_2 + x_3  + x_4 \\
\mbox{sujet à }
& x_1 + 3x_2 + x_4 + s_1 = 4 \\
& 2x_1 +  x_2 + x_4 + s_2 = 3 \\
& x_2 + 4x_3 + x_4 + s_3 = 3 \\
& x \geq 0,\ s \geq 0,
\end{align*}
où $s$ est un vecteur de variables d'écarts.

Comme $z$ est bornée inférieurement par 0, et $z = 0$ si $x = 0$, on voit directement que $x = 0$ est solution optimale, avec $s = (4, 3, 3)$. La base optimale est formée des trois dernières colonnes de la matrice de contraintes du problème sous forme standard, ce qui n'est rien d'autre que la matrice identité.

Tant que $b$ et $c$ restent non-négatifs, $x = 0$ reste solution optimale. Il suit immédiatement que change $b$ en $b + \Delta b$ ou $c$ en $c + \Delta c$ ne changera pas la base optimale si $\Delta b \geq (-4, -3, -3)$ et $\Delta c \geq (-2, -4, -1, -1)$, où les inégalités doivent se lire composante par composante.

Dès lors, pour des petits changements dans $b$ et/ou $c$, $x = 0$ reste optimal, et la valeur optimale reste également inchangée, avec la valeur 0.